<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim - Desafío 2

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.



## Resolucion

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos El libro de Herman Melville, Moby dick, publicada en 1851. Narra la travesía del barco ballenero Pequod, comandado por el capitán Ahab, junto a Ismael y el arponero Queequeg en la obsesiva y autodestructiva persecución de un gran cachalote blanco.

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('moby_dick.txt', sep='/n', header=None, engine='python')
df.head()

,0
0,"The Project Gutenberg eBook of Moby Dick; Or, ..."
1,This ebook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 19222


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
sentence_tokens

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'moby',
  'dick',
  'or',
  'the',
  'whale'],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and'],
 ['most',
  'other',
  'parts',
  'of',
  'the',
  'world',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions'],
 ['whatsoever',
  'you',
  'may',
  'copy',
  'it',
  'give',
  'it',
  'away',
  'or',
  're',
  'use',
  'it',
  'under',
  'the',
  'terms'],
 ['of',
  'the',
  'project',
  'gutenberg',
  'license',
  'included',
  'with',
  'this',
  'ebook',
  'or',
  'online'],
 ['at',
  'www',
  'gutenberg',
  'org',
  'if',
  'you',
  'are',
  'not',
  'located',
  'in',
  'the',
  'united',
  'states'],
 ['you',
  'will',
  'have',
  'to',
  'check',
  'the',
  'laws',
  'of',
  'the',
  'country',
  'where',
  'you',
  'are',
  'located'],
 ['before', 'using', 'this', 'ebook'],
 ['title', 'moby', 'dick', 'or'

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'moby',
  'dick',
  'or',
  'the',
  'whale'],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=4,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 19222


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4383


In [13]:
w2v_model.wv.index_to_key[:10]

['the', 'of', 'and', 'a', 'to', 'in', 'that', 'his', 'it', 'i']

### 3 - Entrenar embeddings

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()],
                 )

Loss after epoch 0: 457665.9375
Loss after epoch 1: 373571.875
Loss after epoch 2: 348439.5625
Loss after epoch 3: 323393.375
Loss after epoch 4: 312875.875
Loss after epoch 5: 307799.625
Loss after epoch 6: 279515.75
Loss after epoch 7: 273548.0
Loss after epoch 8: 271342.0
Loss after epoch 9: 271492.25
Loss after epoch 10: 265168.5
Loss after epoch 11: 264356.5
Loss after epoch 12: 262813.0
Loss after epoch 13: 251180.25
Loss after epoch 14: 236176.0
Loss after epoch 15: 235825.0
Loss after epoch 16: 238149.0
Loss after epoch 17: 229782.5
Loss after epoch 18: 234789.5
Loss after epoch 19: 230445.0
Loss after epoch 20: 229271.5
Loss after epoch 21: 228822.5
Loss after epoch 22: 216768.5
Loss after epoch 23: 222060.5
Loss after epoch 24: 221689.5
Loss after epoch 25: 216822.5
Loss after epoch 26: 217715.0
Loss after epoch 27: 217347.0
Loss after epoch 28: 210406.5
Loss after epoch 29: 216959.5
Loss after epoch 30: 214839.5
Loss after epoch 31: 215939.5
Loss after epoch 32: 207254.5
Los

(28254876, 44135600)

### 4 - Ensayar

Ya entrenamos nuestro modelo y lo ajustamos a 2 palabras de contexto antes y despues de la palabra target.

Como estamos usando Skipgram (predice las palabras de contexto con respecto a una palabra en especifico) vamos a ver qué tan acertados son nuestro modelo y embeddings. 

Comencemos con unas palabras que podriamos esperar del libro, como el nombre del 'Villano' principal, Moby Dick.

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["moby"], topn=10)

[('dick', 0.730833113193512),
 ('earlier', 0.34218069911003113),
 ('mile', 0.3284761607646942),
 ('gleamed', 0.320654958486557),
 ('castaway', 0.31687161326408386),
 ('“do', 0.3128441274166107),
 ('silvery', 0.3122445046901703),
 ('bud', 0.31169506907463074),
 ('dreadful', 0.3070196211338043),
 ('surprised', 0.30541741847991943)]

Probemos con whale

In [26]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(positive=["whale"], topn=10)

[('“hast', 0.3404150903224945),
 ('sperm', 0.3199629783630371),
 ('killer', 0.2904263138771057),
 ('technically', 0.2887957990169525),
 ('v', 0.28635677695274353),
 ('duodecimo', 0.2862356901168823),
 ('other’s', 0.28352808952331543),
 ('sunda', 0.2830929458141327),
 ('heedful', 0.28112998604774475),
 ('owen', 0.2765275537967682)]

Usando 'whale' tambien podemos ver cómo hay ciertos resultados que tienen sentido, como sperm y killer, pero las demás palabras son poco familiares con la palabra en cuestion.

In [24]:
# Ensayar con los ejemplos negativos de una palabras:
w2v_model.wv.most_similar(negative=["moby"])

[('knee', 0.05536611005663872),
 ('memory', 0.05325371026992798),
 ('city', 0.038992054760456085),
 ('completely', 0.03269893676042557),
 ('pitching', 0.02654123865067959),
 ('warm', 0.020307442173361778),
 ('friend', 0.017433514818549156),
 ('planted', 0.017298324033617973),
 ('desert', 0.01601194590330124),
 ('beyond', 0.015878979116678238)]

Podemos ver aca como hay palabras que no están para nada relacionadas con 'moby', tiene sentido porque al tener estas palabras fuera del contexto que elegimos de 2 palabras antes y despues de las palabras leidas por nuestro modelo, lo cual significa que estas palabras son las que menos aparecen en el ambito antes mencionado de 2 palabras.

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_moby = w2v_model.wv.get_vector("moby")
print(vector_moby)

[ 2.27889866e-01  1.49192825e-01 -6.57644868e-01 -2.58743227e-01
 -1.21223897e-01  4.95301366e-01  7.26225555e-01  4.03652281e-01
 -8.28242972e-02 -7.27178752e-01 -5.40994406e-02 -6.75605834e-01
  3.09329748e-01 -1.88476339e-01  2.93105155e-01 -5.67035615e-01
  6.15966737e-01 -2.08326176e-01 -4.27859545e-01  1.03122115e-01
 -7.27898240e-01  6.63431436e-02  1.53949931e-01 -6.24664426e-02
 -7.40783438e-02 -3.61631922e-02 -3.73406619e-01 -1.38633311e-01
  3.42625260e-01 -1.03813156e-01 -3.98082912e-01  1.82548255e-01
  2.23804757e-01 -3.09928358e-01 -5.32241702e-01 -2.70574484e-02
  7.13809311e-01  2.64815420e-01 -6.11282103e-02 -7.55108356e-01
 -2.05957860e-01 -1.09123643e-02 -4.44382668e-01 -1.63952887e-01
  8.73130798e-01  4.73609567e-01 -7.51850232e-02 -1.04510680e-01
 -3.26496065e-01  2.85791308e-01  1.00102432e-01  6.23274505e-01
  3.88598382e-01 -2.37124830e-01 -4.75479275e-01  1.03047371e-01
  6.41192853e-01 -8.08336377e-01  4.60495591e-01  2.88951814e-01
 -2.08778143e-01 -4.33560

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_moby)

[('moby', 1.0),
 ('dick', 0.730833113193512),
 ('earlier', 0.34218069911003113),
 ('mile', 0.3284761607646942),
 ('gleamed', 0.320654958486557),
 ('castaway', 0.31687161326408386),
 ('“do', 0.3128441274166107),
 ('silvery', 0.3122445046901703),
 ('bud', 0.31169506907463074),
 ('dreadful', 0.3070196211338043)]

En forma vectorial vemos el mismo comportamiento donde la palabra dick es la que mas tiene relevancia con la palabra moby.

### 5 - Visualizar agrupación de vectores

In [20]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [21]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() 

Observaciones y conclusiones:

Podemos ver como los vectores de embeddings con palabras muy frecuentemente usadas, por ejemplo preposiciones y sustantivos de tiempo tienen mas relacion entre si, por lo que podriamos inferir que el modelo entrena mejor porque obviamente tiene mas muestras de entrenamiento y aprende mejor las relaciones entre estas palabras.

Dicho esto, anteriormente vimos como hay palabras que si o si deberian ir juntas dada la naturaleza del contenido del libro, como por ejemplo la combinacion 'Moby Dick', da los resultados esperados con un ~70% de certidumbre segun el modelo que entrenamos.

In [22]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() 

Vemos acá en la representacion en 3D del TSNE tambien como lo dicho anteriormente se confirma, en el centro podemos ver como palabras como 'they','as','had','when', etc. Son mas cercanas entre si por el hecho de que tienen mucha mas periodicidad en sus apariciones.

En conclusion, como en los procesos basicos de Redes Neuronales, podria ser beneficioso tunear hiperparametros distintos como ajustar la ventana de contexto(min_count), el numero de negativos(negative), y tambien la dimensionalidad de los vectores(vector_size).

Otro punto a traer a la mesa es el hecho de que el libro de Moby Dick utiliza un vocabulario y narrativa distintos a la literatura moderna, asi que podria ser beneficioso buscar palabras mas acordes a los tiempos en nuestro diccionario de documentos para buscar palabras con mayor similitud.